**Nome:** Rebeca Andrade Medeiros

---


**Matrícula:** 11500782

###Exercício - Redes Neurais

Código teve como base os exemplos 'skipgram.ipynb' e 'rnn.ipynb'

Download do dataset e preparação dos dados

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 14.5 MB/s eta 0:00:00


In [2]:
import datasets
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
import tqdm

In [3]:
noticias = datasets.load_dataset("okite97/news-data")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
noticias

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category'],
        num_rows: 828
    })
})

In [5]:
noticias['train']['Excerpt'][0]

'Uefa has opened disciplinary proceedings against Barcelona, Juventus and Real Madrid over their involvement in the proposed European Super League.'

In [6]:
noticias['train']['Title'][0]

'Uefa Opens Proceedings against Barcelona, Juventus and Real Madrid Over European Super League Plan'

In [7]:
noticias['train']['Category'][0]

'sports'

In [8]:
# Categorias de classificação
noticias.unique('Category')

{'train': ['sports',
  'business',
  'politics',
  'health',
  'tech',
  'entertainment'],
 'test': ['politics', 'business', 'health', 'sports', 'tech', 'entertainment']}

In [9]:
#Divisão do texto em tokens
tokenizer = get_tokenizer('basic_english')

def split_tokens(row):
    row['tokens'] = tokenizer(row['Excerpt'])
    return row

In [10]:
noticias = noticias.map(split_tokens)

Map:   0%|          | 0/4686 [00:00<?, ? examples/s]

Map:   0%|          | 0/828 [00:00<?, ? examples/s]

In [11]:
noticias

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens'],
        num_rows: 828
    })
})

In [12]:
noticias['train']['tokens'][0]

['uefa',
 'has',
 'opened',
 'disciplinary',
 'proceedings',
 'against',
 'barcelona',
 ',',
 'juventus',
 'and',
 'real',
 'madrid',
 'over',
 'their',
 'involvement',
 'in',
 'the',
 'proposed',
 'european',
 'super',
 'league',
 '.']

In [13]:
#Criar vocabulário

vocab_custom = build_vocab_from_iterator(noticias['train']['tokens'],
                                         min_freq=2,
                                         specials=[''])

In [14]:
vocab_custom['uefa']
len(vocab_custom)

6019

In [15]:
vocab_custom.set_default_index(vocab_custom[''])

In [65]:
# Transfomar token -> id
def vocab(tokens):
  result = []
  for token in tokens:
    result.append(vocab_custom[token])

  return result

In [17]:
def token2id(row):
    row['tokens_id'] = vocab(row['tokens'])
    return row

In [18]:
noticias = noticias.map(token2id)

Map:   0%|          | 0/4686 [00:00<?, ? examples/s]

Map:   0%|          | 0/828 [00:00<?, ? examples/s]

In [19]:
noticias

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens', 'tokens_id'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens', 'tokens_id'],
        num_rows: 828
    })
})

In [20]:
noticias['train']['tokens_id'][0]

[577,
 7,
 755,
 4675,
 3008,
 64,
 229,
 2,
 465,
 10,
 210,
 196,
 41,
 36,
 3633,
 6,
 1,
 572,
 178,
 123,
 50,
 9]

In [63]:
# Transformar 'Category' em id's

def split_tokens_category(row):
    row['tokens_category'] = tokenizer(row['Category'])
    return row

noticias = noticias.map(split_tokens_category)

Map:   0%|          | 0/4686 [00:00<?, ? examples/s]

Map:   0%|          | 0/828 [00:00<?, ? examples/s]

In [64]:
noticias

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens', 'tokens_id', 'tokens_category'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens', 'tokens_id', 'tokens_category'],
        num_rows: 828
    })
})

In [66]:
vocab_category = build_vocab_from_iterator(noticias['train']['Category'],
                                         min_freq=1,
                                         specials=[''])

In [67]:
vocab_category.set_default_index(vocab_category[''])

In [72]:
def vocab_category_tokens(tokens):
  result = []
  for token in tokens:
    result.append(vocab_category[token])

  return result

In [76]:
def token2id_category(row):
    row['tokens__category_id'] = vocab(row['tokens_category'])
    return row

In [77]:
noticias = noticias.map(token2id_category)

Map:   0%|          | 0/4686 [00:00<?, ? examples/s]

Map:   0%|          | 0/828 [00:00<?, ? examples/s]

In [78]:
noticias

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens', 'tokens_id', 'tokens_category', 'tokens__category_id'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens', 'tokens_id', 'tokens_category', 'tokens__category_id'],
        num_rows: 828
    })
})

In [80]:
noticias['train'][0]

{'Title': 'Uefa Opens Proceedings against Barcelona, Juventus and Real Madrid Over European Super League Plan',
 'Excerpt': 'Uefa has opened disciplinary proceedings against Barcelona, Juventus and Real Madrid over their involvement in the proposed European Super League.',
 'Category': 'sports',
 'tokens': ['uefa',
  'has',
  'opened',
  'disciplinary',
  'proceedings',
  'against',
  'barcelona',
  ',',
  'juventus',
  'and',
  'real',
  'madrid',
  'over',
  'their',
  'involvement',
  'in',
  'the',
  'proposed',
  'european',
  'super',
  'league',
  '.'],
 'tokens_id': [577,
  7,
  755,
  4675,
  3008,
  64,
  229,
  2,
  465,
  10,
  210,
  196,
  41,
  36,
  3633,
  6,
  1,
  572,
  178,
  123,
  50,
  9],
 'tokens_category': ['sports'],
 'tokens__category_id': [603]}

##### Dataset e Dataloader

In [84]:

class News(Dataset):
  def __init__(self, noticias):
    self.data = noticias['tokens_id']
    self.labels = noticias['tokens__category_id']

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return torch.tensor(self.data[idx]), torch.tensor(self.labels[idx])

In [85]:
news_dataset = News(noticias['train'])

In [86]:
news_dataloader = DataLoader(News(noticias['train']), shuffle = True)

In [87]:
vocab_size = len(vocab_custom)
embedding_size = 32

class ClassificacaoNews(nn.Module):
  def __init__(self, vocab_size, embedding_size):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_size)
    self.x = nn.Linear(embedding_size, 6)
    self.softmax = nn.Softmax()

  def forward(self, input):
    emb = self.embedding(input)
    out = self.softmax(emb)
    return out

In [88]:
modelo = ClassificacaoNews(vocab_size, embedding_size)

In [89]:
modelo(torch.tensor(noticias['train']['tokens_id'][0]))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([[0.0006, 0.2405, 0.0065, 0.0197, 0.0129, 0.0252, 0.0137, 0.0182, 0.0187,
         0.0086, 0.0298, 0.0868, 0.0337, 0.0620, 0.0172, 0.0187, 0.0075, 0.0130,
         0.0398, 0.1241, 0.0258, 0.0192, 0.0064, 0.0117, 0.0357, 0.0168, 0.0203,
         0.0115, 0.0252, 0.0157, 0.0075, 0.0070],
        [0.0392, 0.2018, 0.0682, 0.0441, 0.0055, 0.0036, 0.0030, 0.0094, 0.0092,
         0.0291, 0.0540, 0.0151, 0.0092, 0.0436, 0.0187, 0.0065, 0.0177, 0.0487,
         0.0180, 0.0312, 0.0082, 0.0728, 0.0208, 0.0116, 0.0379, 0.0042, 0.0009,
         0.0183, 0.0143, 0.0218, 0.0762, 0.0371],
        [0.0150, 0.0109, 0.0115, 0.1499, 0.0161, 0.0098, 0.0635, 0.0290, 0.0097,
         0.0101, 0.0130, 0.0222, 0.0326, 0.0449, 0.0086, 0.0113, 0.0082, 0.0337,
         0.0582, 0.0472, 0.0087, 0.1184, 0.0087, 0.0180, 0.0071, 0.0123, 0.0332,
         0.0429, 0.0336, 0.0548, 0.0106, 0.0463],
        [0.0382, 0.0036, 0.0123, 0.0038, 0.0091, 0.0067, 0.0259, 0.1053, 0.0096,
         0.0235, 0.0097, 0.0078, 0.0225,

**Treinamento**

In [90]:
device = torch.device('cuda')
modelo.to(device)

ClassificacaoNews(
  (embedding): Embedding(6019, 32)
  (x): Linear(in_features=32, out_features=6, bias=True)
  (softmax): Softmax(dim=None)
)

In [91]:
lr = 1e-5
epochs = 3

In [92]:
loss = nn.BCELoss()
optimizer = torch.optim.Adam(modelo.parameters(), lr=lr)

In [93]:
def train(dataloader):
  size = len(news_dataloader.dataset)
  loss_treino = 0

  for Excerpt, Category in enumerate(news_dataloader):
    Excerpt, Category = Excerpt.to(device), Category.to(device)

    out = modelo(Excerpt[0])
    print("Out:", out)

    loss = loss(out.to(torch.float32), Category.to(torch.float32))
    print("Loss:", loss)

    loss_treino = loss_treino + loss.item
    print("Loss treino:", loss_treino)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  loss_treino = loss_treino/size

  return loss_treino


In [94]:
size = len(news_dataloader.dataset)

In [95]:
loss_treinamento = []
loss_teste = []

for a in range(epochs):
  print("Epoch: ", a)

  print("Treinamento")
  loss_treinamento.append(train(news_dataloader))

Epoch:  0
Treinamento


AttributeError: ignored